In [1]:
##%pip install ipynb

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ipynb.fs.full.twscrape import *
import pandas as pd

In [3]:
user_name = input("Input the username: ")
limit = int(input("Input the amount of tweets to get: "))
result = await gettweets(user_name, limit)
##print(result)

2023-11-08 08:43:30.234 | WARNING  | twscrape.accounts_pool:add_account:76 - Account BingBong2397074 already exists
2023-11-08 08:43:30.265 | WARNING  | twscrape.accounts_pool:add_account:76 - Account BBong20237297 already exists
2023-11-08 08:43:30.419 | INFO     | twscrape.accounts_pool:login_all:155 - [1/1] Logging in BBong20237297 - twitterscrapertest@rambler.ua
2023-11-08 08:43:32.159 | INFO     | twscrape.accounts_pool:login:138 - Logged in to BBong20237297 successfully


user login: User(id=813286, id_str='813286', url='https://twitter.com/BarackObama', username='BarackObama', displayname='Barack Obama', rawDescription='Dad, husband, President, citizen.', created=datetime.datetime(2007, 3, 5, 22, 8, 25, tzinfo=datetime.timezone.utc), followersCount=132022509, friendsCount=549115, statusesCount=16955, favouritesCount=8, listedCount=215262, mediaCount=2592, location='Washington, DC', profileImageUrl='https://pbs.twimg.com/profile_images/1329647526807543809/2SGvnHYV_normal.jpg', profileBannerUrl='https://pbs.twimg.com/profile_banners/813286/1633800288', protected=None, verified=False, blue=True, blueType=None, descriptionLinks=[TextLink(url='http://obamabook.com', text='obamabook.com', tcourl='https://t.co/kHvnxozw8x')], _type='snscrape.modules.twitter.User') 

user id: 813286 

[Tweet(id=1721947245203894563, id_str='1721947245203894563', url='https://twitter.com/BarackObama/status/1721947245203894563', date=datetime.datetime(2023, 11, 7, 17, 46, 30, tzin

In [4]:
##type(result)

In [5]:
df_tweets = pd.DataFrame(result)

In [6]:
##df_tweets.head()

In [7]:
##for column in df_tweets.columns:
##    print(column, " ", type(column))

In [8]:
## avoid sql keywords
df_tweets.rename(columns={'url': 'tweet_url', 'date': 'publish_date','user': 'tweet_user'}, inplace=True)

In [9]:
##df_tweets.head()

In [10]:
##%pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 2.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.9-cp311-cp311-linux_x86_64.whl size=490591 sha256=b4177871be1f4e4636967549e2c127c7c4ebd17a6617ee79abd66b033b0e51d3
  Stored in directory: /home/vscode/.cache/pip/wheels/ab/34/b9/78ebef1b3220b4840ee482461e738566c3c9165d2b5c914f51
Successfully built psycopg2

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import psycopg2

In [12]:
conn = psycopg2.connect(dbname="postgres",user="postgres", password="postgres", port="5432", host="localhost")
conn.autocommit = True
cursor = conn.cursor()

In [13]:
import sys
import traceback

In [88]:
## selection of usernames from mentionedUsers

##i = 0
##for index, row in df_tweets.iterrows():
##    i=i+1
##    (print(list(pd.DataFrame(row.mentionedUsers)['username'])) if str(row.mentionedUsers) != '[]' else print(str(row.mentionedUsers)))
##    if i == 16:
##        break    

[]
[]
[]
['POTUS']
['MichelleObama']
[]
[]
['ObamaFoundation']
['ObamaFoundation']
['BarackObama']
['baratunde']
[]
['ObamaFoundation']
['POTUS']
[]
['ObamaFoundation', 'BarackObama']


In [96]:
i = 0
for index, row in df_tweets.iterrows():
    i=i+1
    ##(print(list(pd.DataFrame(row.mentionedUsers)['username'])) if str(row.mentionedUsers) != '[]' else print(str(row.mentionedUsers)))
    print(row.retweetedTweet['rawContent']) if str(row.retweetedTweet) != None else print(row.retweetedTweet)
    if i == 6:
        break    

TypeError: 'NoneType' object is not subscriptable

In [92]:
for index, row in df_tweets.iterrows():
    try:
        cursor.execute('''INSERT INTO Tweets (
                            id,
                            id_str,
                            tweet_url,
                            publish_date,
                            tweet_user,
                            lang,
                            rawcontent,
                            replycount,
                            retweetcount,
                            likecount,
                            quotecount,
                            conversationid,
                            hashtags,
                            cashtags,
                            mentionedusers,
                            links,
                            viewcount,
                            retweetedtweet,
                            quotedtweet,
                            place,
                            coordinates,
                            inreplytotweetid,
                            inreplytouser,
                            source,
                            sourceurl,
                            sourcelabel,
                            media,
                            _type
                        )
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);''', 
                       (row.id,
                        row.id_str,
                        row.tweet_url,
                        row.publish_date,
                        str(row.tweet_user['username']),
                        row.lang,
                        row.rawContent,
                        row.replyCount,
                        row.retweetCount,
                        row.likeCount,
                        row.quoteCount,
                        row.conversationId,
                        row.hashtags,
                        row.cashtags,
                        (list(pd.DataFrame(row.mentionedUsers)['username']) if str(row.mentionedUsers) != '[]' else row.mentionedUsers),
                        str(row.links),
                        row.viewCount,
                        str(row.retweetedTweet),
                        str(row.quotedTweet),
                        str(row.place),
                        str(row.coordinates),
                        str(row.inReplyToTweetId),
                        str(row.inReplyToUser),
                        str(row.source),
                        str(row.sourceUrl),
                        str(row.sourceLabel),
                        str(row.media),
                        str(row._type)
                       )
        )
        
    except psycopg2.errors.UniqueViolation:
        print('Tweet already in database. Tweet ID: ', row.id)
    except BaseException as ex:
            ex_type, ex_value, ex_traceback = sys.exc_info()
            
            # Extract unformatter stack traces as tuples
            trace_back = traceback.extract_tb(ex_traceback)

            # Format stacktrace
            stack_trace = list()

            for trace in trace_back:
                stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))
            print('Error at: ', row.id, 'Error: ', ex_value)
            print("Exception type : %s " % ex_type.__name__)
            print("Exception message : %s" %ex_value)
            print("Stack trace : %s" %stack_trace)
            print('______________')

## adjust attribute types for lists to sql array as done for mentionedUsers

Tweet already in database. Tweet ID:  1721947245203894563
Tweet already in database. Tweet ID:  1721597427818410471
Tweet already in database. Tweet ID:  1706329460973490375
Tweet already in database. Tweet ID:  1721177408869404749
Tweet already in database. Tweet ID:  1720943828738183412
Tweet already in database. Tweet ID:  1720916340276433010
Tweet already in database. Tweet ID:  1720902753881366930
Tweet already in database. Tweet ID:  1720615585950625825
Tweet already in database. Tweet ID:  1720614862684901770
Tweet already in database. Tweet ID:  1720594715274363014
Tweet already in database. Tweet ID:  1720560650139599230
Tweet already in database. Tweet ID:  1720524872474853672
Tweet already in database. Tweet ID:  1720523848867553335
Tweet already in database. Tweet ID:  1720458237315551378
Tweet already in database. Tweet ID:  1720458262523326961
Tweet already in database. Tweet ID:  1720456124653015483
Tweet already in database. Tweet ID:  1720092245586809232
Tweet already 